# Metrics exploration

This notebook emphasizes the text of some metrics for the QA statement relying only in (question, answer, prediction) pairs.


## Libs and Variables

In [1]:
import re
import random
import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt

clean_repeated_ws = lambda text: re.sub(r"\s+", " ", text).strip()

In [2]:
def seed_all():
    """
    Seed all the random number generators to ensure reproducibility.

    This function sets the seed for the Python built-in random module and the NumPy random module to 42.
    This ensures that any random operations performed using these modules will produce the same results
    each time the code is run.
    """
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)


seed_all()

In [3]:
DATA_PATH = "../data/data.csv"

## Load data

In [4]:
data = pd.read_csv(DATA_PATH)

data["question"] = data["question"].astype(str)
data["answer"] = data["answer"].astype(str)

data["question"] = data["question"].apply(clean_repeated_ws)
data["answer"] = data["answer"].apply(clean_repeated_ws)

In [5]:
data.sample(5).style

,question,answer
3634,Is Juvenile Huntington disease inherited ?,"How is Juvenile Huntington disease (HD) inherited? Juvenile HD is inherited in an autosomal dominant manner, which means that one copy of the altered gene in each cell is sufficient to cause the disorder. An affected person usually inherits the altered gene from one affected parent. As the altered HTT gene is passed from one generation to the next, the size of the CAG trinucleotide repeat often increases in size. A larger number of repeats is usually associated with an earlier onset of signs and symptoms (anticipation). A larger number of repeats is usually associated with an earlier onset of signs and symptoms. Most often, children with juvenile HD inherit the expanded CAG trinucleotide repeat from their fathers, although on occasion they inherit it from their mothers."
13745,What are the treatments for nemaline myopathy ?,These resources address the diagnosis or management of nemaline myopathy: - Gene Review: Gene Review: Nemaline Myopathy - Genetic Testing Registry: Nemaline myopathy - Genetic Testing Registry: Nemaline myopathy 1 - Genetic Testing Registry: Nemaline myopathy 10 - Genetic Testing Registry: Nemaline myopathy 2 - Genetic Testing Registry: Nemaline myopathy 3 - Genetic Testing Registry: Nemaline myopathy 4 - Genetic Testing Registry: Nemaline myopathy 5 - Genetic Testing Registry: Nemaline myopathy 6 - Genetic Testing Registry: Nemaline myopathy 7 - Genetic Testing Registry: Nemaline myopathy 8 - Genetic Testing Registry: Nemaline myopathy 9 These resources from MedlinePlus offer information about the diagnosis and management of various health conditions: - Diagnostic Tests - Drug Therapy - Surgery and Rehabilitation - Genetic Counseling - Palliative Care
4395,What is (are) Hailey-Hailey disease ?,"Hailey-Hailey disease is a hereditary blistering skin disease. Signs and symptoms include a painful rash and blistering in skin folds such as the armpits, groin, neck, under the breasts, and between the buttocks. Secondary bacterial infections are not uncommon. Symptoms are often worse in summer months due to heat, sweating and friction. Hailey-Hailey disease is caused by mutations in the ATP2C1 gene and is inherited in an autosomal dominant manner. Treatment focuses on reducing symptoms and preventing flares."
10274,Is transthyretin amyloidosis inherited ?,"This condition is inherited in an autosomal dominant pattern, which means one copy of the altered gene in each cell is sufficient to cause the disorder. In most cases, an affected person inherits the mutation from one affected parent. Rarely, cases result from new mutations in the gene and occur in people with no history of the disorder in their family. Not all people who have a TTR gene mutation will develop transthyretin amyloidosis."
149,How many people are affected by Balance Problems ?,"In 2008, an estimated 14.8 percent of American adults (33.4 million) had a balance or dizziness problem during the past year. See statistics about the frequency of balance and other sensory impairments in older adults. (Centers for Disease Control and Prevention)"


## Metrics

In [46]:
import evaluate

sentence1 = "the cat ate rat"
sentence2 = "the cat ate rat"

In [53]:
import nltk


BLEUscore = nltk.translate.bleu_score.sentence_bleu(
    ["the cat eat the rat", "eat the rat"], "eat the rat"
)
print(f"BLEUscore: {BLEUscore:.4f}%")

BLEUscore: 1.0000%


In [28]:
bleu = evaluate.load("google_bleu")
result = bleu.compute(predictions=[sentence1], references=[[sentence2]])
print(result)

{'google_bleu': 0.6}


In [29]:
bertscore = evaluate.load("bertscore")
result = bertscore.compute(
    predictions=[sentence1],
    references=[[sentence2]],
    lang="en",
    model_type="distilbert-base-uncased",
)
print(result)

{'precision': [0.9554272294044495], 'recall': [0.9554272294044495], 'f1': [0.9554272294044495], 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.46.3)'}


In [55]:
from ragas import SingleTurnSample
from ragas.metrics import BleuScore

test_data = {
    "user_input": "summarise given text\nThe company reported an 8% rise in Q3 2024, driven by strong performance in the Asian market. Sales in this region have significantly contributed to the overall growth. Analysts attribute this success to strategic marketing and product localization. The positive trend in the Asian market is expected to continue into the next quarter.",
    "response": "The company experienced an 8% increase in Q3 2024, largely due to effective marketing strategies and product adaptation, with expectations of continued growth in the coming quarter.",
    "reference": "The company reported an 8% growth in Q3 2024, primarily driven by strong sales in the Asian market, attributed to strategic marketing and localized products, with continued growth anticipated in the next quarter.",
}
metric = BleuScore()
test_data = SingleTurnSample(**test_data)
metric.single_turn_score(test_data)

0.13718598426177148